In [1]:
from scipy.linalg import svd
import time
import scipy
import random
import portpy.photon as pp
import numpy as np
from cvxpy import *
from scipy import sparse
import sklearn.metrics
import cProfile
import time
import sys
import psutil

In [3]:
data_dir = r'../data'
data = pp.DataExplorer(data_dir=data_dir)
patient_id = 'Lung_Phantom_Patient_1'
data.patient_id = patient_id
ct = pp.CT(data)
structs = pp.Structures(data)
beams = pp.Beams(data)
opt_params = data.load_config_opt_params(protocol_name='Lung_2Gy_30Fx')
structs.create_opt_structures(opt_params)
inf_matrix_sparse = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams)
protocol_name = 'Lung_2Gy_30Fx'
clinical_criteria = pp.ClinicalCriteria(data, protocol_name)
plan_sparse = pp.Plan(ct, structs, beams, inf_matrix_sparse, clinical_criteria)
opt = pp.Optimization(plan_sparse, opt_params=opt_params)
opt.create_cvxpy_problem()

sol_sparse = opt.solve(solver='MOSEK', verbose=True)
dose_sparse_1d = plan_sparse.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions())
x_sparse =sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions()

beams_full = pp.Beams(data, load_inf_matrix_full=True)
inf_matrix_full = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams_full, is_full=True)
plan_full = pp.Plan(ct, structs, beams, inf_matrix_full, clinical_criteria)
dose_full_1d = plan_full.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_full.get_num_of_fractions())

A_full = plan_full.inf_matrix.A
A_sparse = plan_sparse.inf_matrix.A


creating rinds.. This step may take some time due to dilation
rinds created!!
Creating BEV..
Loading sparse influence matrix...
Done
Objective Start
Objective done
Constraints Start
Structure ESOPHAGUS not available!
Structure ESOPHAGUS not available!
Constraints done
                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Jan 27 08:12:06 PM: Your problem has 1946 variables, 14 constraints, and 0 parameters.
(CVXPY) Jan 27 08:12:06 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 27 08:12:06 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 27 08:12:06 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 27 08:12:06 PM: Your problem is compiled with the CPP canonicalization backend.
----------------------------------------

(CVXPY) Jan 27 08:12:10 PM:   Primal.  obj: 4.2045390026e+01    nrm: 1e+02    Viol.  con: 2e-06    var: 5e-07    cones: 0e+00  
(CVXPY) Jan 27 08:12:10 PM:   Dual.    obj: 4.2045390059e+01    nrm: 6e+03    Viol.  con: 0e+00    var: 9e-09    cones: 0e+00  
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Jan 27 08:12:10 PM: Problem status: optimal
(CVXPY) Jan 27 08:12:10 PM: Optimal value: 4.205e+01
(CVXPY) Jan 27 08:12:10 PM: Compilation took 1.343e+00 seconds
(CVXPY) Jan 27 08:12:10 PM: Solver (including time spent in interface) took 2.263e+00 seconds
Creating BEV..
Loading full influence matrix..
Done


مقاله ی ۱۵۰۱.۰۱۷۱۱

In [4]:
start_time = time.time()

def frequent_directions(l, A_full):
    n, d = (A_full.T).shape
    #l = int(np.ceil(1 / epsilon)) 
    B = np.zeros((l, d))
    
    for i in range(n):
        Bl = A_full[i]
        U, Sigma, Vt = svd(B, full_matrices=False)
        #l = min(l, d)
        delta = Sigma ** 2
        #B = np.vstack((B[1:], np.zeros((1, d))))
        B = np.sqrt(Sigma ** 2 - delta * np.identity(l)) @ Vt
        #C += np.diag(Sigma) @ Vt  
        
    return B

n, d = (A_full.T).shape
l = n  #l   فقط 542 میشه. 60183 نمیشه! را چند بگیرم که مناسب باشه؟
B = frequent_directions(l, A_full)
print(B)  

end_time = time.time()

total_execution_time = end_time - start_time
print(f"زمان کل اجرا: {total_execution_time} ثانیه")


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
زمان کل اجرا: 1489.8521308898926 ثانیه


In [6]:
## بررسی  پراکندگی ماتریس

is_sparse = sparse.issparse(B)
is_sparse

False

In [41]:
B = B
A = A_full
d = dose_full_1d
print(f'B = {B.shape}\nA = {A.shape}\nd = {d.shape}')

B = (542, 60183)
A = (60183, 542)
d = (60183,)


In [42]:
start_time = time.time()

sol_ls_B = scipy.sparse.linalg.lsqr(B @ A , B @ d)   

end_time = time.time()

total_execution_time = end_time - start_time
print(f"زمان کل اجرا: {total_execution_time} ثانیه")

زمان کل اجرا: 0.6757786273956299 ثانیه


In [44]:
sol_ls_B

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 

In [43]:

sol_ls_A_full = scipy.sparse.linalg.lsqr(A , d)

x_A_full = sol_ls_A_full[0]

cost = cp.norm(A @ x_A_full - A @ x_Fre, "fro")/ cp.norm(A @ x_A_full, "fro")


# تغییر مسئله بهینه‌سازی
prob = cp.Problem(cp.Minimize(cost))

# حل مسئله
prob.solve()


1.0

In [33]:
import numpy as np

def calculate_absolute_error(vector1, vector2):
    array1 = np.array(x_A_full)
    array2 = np.array(x_Fre)

    absolute_error = np.linalg.norm(array1 - array2, ord=1) 

    return absolute_error


error = calculate_absolute_error(x_A_full, x_Fre)
print(f"نرم خطای مطلق بین دو بردار: {error}")


نرم خطای مطلق بین دو بردار: 4095.8777787239264


In [24]:
import cvxpy as cp
x_Fre = sol_ls_B[0]
print(f'x_Fre :\t {x_Fre[0:6]}')


cost = cp.sum_squares((A @ x_A_full  - A @ x_Fre  )/ cp.norm(A @ x_A_full))
prob = cp.Problem(Minimize(cost))
prob.solve()

x_Fre :	 [0. 0. 0. 0. 0. 0.]


1.0000000000000002

In [34]:
memory_usage_sol_ls_B = sys.getsizeof(sol_ls_B )
megabyte_size_sol_ls_B = memory_usage_sol_ls_B / 1024 / 1024

print(f"میزان حافظه sol_ls_JL_1 : {megabyte_size_sol_ls_B} مگابایت")

میزان حافظه sol_ls_JL_1 : 0.00011444091796875 مگابایت


In [35]:
memory_usage_B = sys.getsizeof(B )
megabyte_size_B = memory_usage_B / 1024 / 1024

print(f"میزان حافظه ساخت JL_2 : {megabyte_size_B} مگابایت")

میزان حافظه ساخت JL_2 : 0.0001220703125 مگابایت


In [36]:
## MSE = Mean Squared Error

d_true = d          # دوز واقعی
d_pred = A @ x_Fre   # دوز پیش‌بینی شده

MSE = np.mean((d_true - d_pred) ** 2)
print("Mean Squared Error :", MSE)


Mean Squared Error : 191.53833375899814


In [40]:
## تابع (هدف)خطای میانگین مربعات
## MSE = Mean Squared Error

d_true = A @ x_A_full        # دوز واقعی
d_pred = A @ x_Fre   # دوز پیش‌بینی شده

MSE = np.mean((d_true - d_pred) ** 2)
print("Mean Squared Error :", MSE)


Mean Squared Error : 191.5383338107377


In [27]:
import numpy as np

def calculate_relative_error(vector1, vector2):
   
    array1 = np.array(x_A_full)
    array2 = np.array(x_Fre)

    norm_vector2 = np.linalg.norm(x_A_full)

    relative_error = np.linalg.norm(x_A_full - x_Fre) / norm_vector2

    return relative_error

error = calculate_relative_error(x_A_full, x_Fre)
print(f"نرم خطای نسبی بین دو بردار: {error}")


نرم خطای نسبی بین دو بردار: 1.0


In [25]:
cost = cp.sum_squares((A @ x_Fre - d )/ cp.norm(d))
prob = cp.Problem(Minimize(cost))
prob.solve()

1.0

In [30]:
r1 = np.linalg.matrix_rank(B @ A_full ) 
r2 = np.linalg.matrix_rank(A_full)      
print( f' رتبه ماتریس تقریب = {r1}\nرتبه ماتریس کامل = {r2}')

 رتبه ماتریس تقریب = 0
رتبه ماتریس کامل = 542


In [18]:
nonzero_count = np.count_nonzero(B)
print("تعداد مقادیر غیر صفر ماتریس کامل: ", nonzero_count)
num_non_zero = np.count_nonzero(B @ A)
print("تعداد عناصر غیر صفر در ماتریس BA :", num_non_zero)

تعداد مقادیر غیر صفر ماتریس کامل:  0
تعداد عناصر غیر صفر در ماتریس BA : 0


dose_sparse_1d.shape